In [ ]:
## This script extracts deep features from LFDP patches

In [ ]:
# import packages
import os, cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
## packages for deep learning
import tensorflow as tf
from keras.applications import resnet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from keras.models import Model
from tensorflow.python.client import device_lib
## check GPU
print(device_lib.list_local_devices())

In [ ]:
# load shadow-free image
img = cv2.imread('../output/images/shadow_free.jpg')
## transform color channel
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# load the pretrained cnn model ResNet-50
base_model = resnet50.ResNet50(weights='imagenet')
resnet_model = Model(inputs=base_model.input,
                    outputs=base_model.get_layer('avg_pool').output)

# extract features, and save in the feat_cnn.csv file
## set resolution and window size
rs = 100 ## resolution 
size_wind = 100 # window size 
num_row = int(img.shape[0]/rs)
num_col = int(img.shape[1]/rs)
## extract features
csvPath = '../output/features/feat_cnn.csv'
f = open(csvPath, 'w')
x_img = np.ones((num_col, 224, 224, 3))
print('...Start Feature Extraction...')
time_start = datetime.now()
for i in range(num_row):
    if (i+1) % 20 == 0:
        print('Finished:', "{:.1%}".format((i+1) / num_row), end="\r")
    for j in range(num_col):
        r_1 = int(np.max([0, i*rs + rs/2 - size_wind/2]))
        r_2 = int(np.min([img.shape[0], i*rs + rs/2 +  size_wind/2]))
        c_1 = int(np.max([0, j*rs + rs/2 - size_wind/2]))
        c_2 = int(np.min([img.shape[1], j*rs + rs/2 + size_wind/2]))
        img_patch = img[r_1:r_2, c_1:c_2]
        img_resize = cv2.resize(img_patch, (224, 224))
        x_img[j] = img_resize
    x_img = preprocess_input(x_img)
    preds = resnet_model.predict(x_img)
    # save features line by line
    for j in range(num_col):
        str1 = list(preds[j])
        line = ', '.join(list(map(str, str1))) + '\n'
        f.write(line)        
f.close()
print('Time for feature extraction:', datetime.now()-time_start)